In [ ]:
!conda create -n qiskit-env python=3.10
!conda activate qiskit-env


Retrieving notices: done
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/qiskit-env

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libcxx-17.0.6              |       he5c5206_4         489 KB
    ncurses-6.5                |       hee39554_0         886 KB
    openssl-3.0.17             |       h4ee41c1_0         4.3 MB
    python-3.10.18             |       h19e8193_0        12.9 MB
    sqlite-3.50.2              |       h79febb2_1         1.0 MB
    tk-8.6.14                  |       h6ba3021_1         3.3 MB
    ------------------------------------------------------------
                                           Total:        22.8 MB

The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/osx-arm64::bzip2-1.0.8-h80987f9_6 
  ca-ce

In [1]:
import numpy as np
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SLSQP

def create_deuteron_hamiltonian(N, hbar_omega=7.0, V0=-5.68658111):
    hamiltonian_terms = {}
    for m in range(N):
        for n in range(N):
            label = f"+_{m} -_{n}"
            kinetic = (hbar_omega/2) * (
                (2*m + 1.5)*int(m==n)
                - np.sqrt(m*(m+0.5))*int(m==n+1)
                - np.sqrt((m+1)*(m+1.5))*int(m==n-1)
            )
            potential = V0 * int(m==0 and n==0) * int(m==n)
            hamiltonian_terms[label] = kinetic + potential
    fermion_op = FermionicOp(hamiltonian_terms, num_spin_orbitals=N)
    mapper = JordanWignerMapper()
    qubit_ham = mapper.map(fermion_op)
    return qubit_ham

# Backend setup
backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend)

# Create Hamiltonians
qubit_hamiltonians = [create_deuteron_hamiltonian(N) for N in range(1, 4)]

# VQE loop
for i, H in enumerate(qubit_hamiltonians, start=1):
    num_qubits = H.num_qubits
    ansatz = TwoLocal(num_qubits, ['rz', 'ry'], 'cx', reps=i)
    vqe = VQE(ansatz=ansatz, optimizer=SLSQP(), quantum_instance=qi)
    result = vqe.compute_minimum_eigenvalue(H)
    print(f"H_{i} binding energy ≈ {result.eigenvalue.real:.6f} MeV")


ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.